<a href="https://colab.research.google.com/github/Fackor/galaxy_zoo_classification_project/blob/master/Galaxy_zoo_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
!pip install -q kaggle

In [2]:
%cd ../root

/root


In [4]:
!mkdir .kaggle
%cd .kaggle
uploading = files.upload()

/root/.kaggle


Saving kaggle.json to kaggle.json


In [5]:
%cd ../../content

/content


In [6]:
!kaggle competitions download -c galaxy-zoo-the-galaxy-challenge

  0% 0.00/265k [00:00<?, ?B/s]
100% 265k/265k [00:00<00:00, 37.2MB/s]
  0% 0.00/265k [00:00<?, ?B/s]
100% 265k/265k [00:00<00:00, 86.4MB/s]
  0% 0.00/520k [00:00<?, ?B/s]
100% 520k/520k [00:00<00:00, 68.9MB/s]
 97% 769M/792M [00:09<00:00, 131MB/s]
100% 792M/792M [00:09<00:00, 88.6MB/s]
100% 1.00G/1.01G [00:17<00:00, 39.8MB/s]
100% 1.01G/1.01G [00:17<00:00, 62.8MB/s]
  0% 0.00/4.63M [00:00<?, ?B/s]
100% 4.63M/4.63M [00:00<00:00, 75.0MB/s]


In [7]:
!unzip training_solutions_rev1.zip
!unzip images_training_rev1.zip

Archive:  training_solutions_rev1.zip
  inflating: training_solutions_rev1.csv  
Archive:  images_training_rev1.zip
   creating: images_training_rev1/
  inflating: images_training_rev1/100008.jpg  
  inflating: images_training_rev1/100023.jpg  
  inflating: images_training_rev1/100053.jpg  
  inflating: images_training_rev1/100078.jpg  
  inflating: images_training_rev1/100090.jpg  
  inflating: images_training_rev1/100122.jpg  
  inflating: images_training_rev1/100123.jpg  
  inflating: images_training_rev1/100128.jpg  
  inflating: images_training_rev1/100134.jpg  
  inflating: images_training_rev1/100143.jpg  
  inflating: images_training_rev1/100150.jpg  
  inflating: images_training_rev1/100157.jpg  
  inflating: images_training_rev1/100187.jpg  
  inflating: images_training_rev1/100204.jpg  
  inflating: images_training_rev1/100237.jpg  
  inflating: images_training_rev1/100259.jpg  
  inflating: images_training_rev1/100263.jpg  
  inflating: images_training_rev1/100288.jpg  
  i

-----------------------------------------------------------------------------
Beginning preprocessing, developing and training the ResNet model for the task of classification

In [8]:
#Importing the libraries
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, ZeroPadding2D, Flatten, AveragePooling2D, MaxPooling2D, Conv2D, Activation, BatchNormalization, GlobalAveragePooling2D, GlobalMaxPooling2D, Dense, Dropout
from keras.activations import relu, softmax
from keras.models import Model, load_model
from keras import regularizers
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.initializers import glorot_uniform
from keras.callbacks import ModelCheckpoint, TensorBoard

import keras.backend as K
import os
from sklearn.model_selection import train_test_split
import gc

from matplotlib.pyplot import imshow
%matplotlib inline

Using TensorFlow backend.


In [0]:

'''
This part processes the images by first cropping them to 312x312 size and then resizing them
to 224x224 images. Here we maintain a label for each image that will be used later to help split data into 
training and test sets.
'''
set_size = 10000
X_full = {}

Y_full = pd.read_csv('training_solutions_rev1.csv', nrows=set_size)

i = 0
for id in Y_full["GalaxyID"].tolist():
  if i > set_size:
    break
  name = 'images_training_rev1/'+str(id)+'.jpg'
  img = cv2.imread(name)
  img = img[56:368, 56:368]
  img = cv2.resize(img, (224, 224))
  X_full[name] = img
  i += 1


In [0]:
'''
This part conttains the declarations for the split_ratio variable,
which controls the ratio in which the training and test are split
'''   
split_ratio = 0.8


'''
This part reads in the csv files and splits it into training and test sets, we use the train_test_split() method from 
the sklearn module.
'''

Y_train, Y_test = train_test_split(Y_full, test_size=(1 - split_ratio))
del Y_full
gc.collect()

'''
We now extract the GalaxyID labels from the Y_train and Y_test dataframes and use these labels to create our 
X_train and X_test variables containg the images.
'''
train_id = Y_train["GalaxyID"].tolist()
test_id = Y_test["GalaxyID"].tolist()


In [11]:
train_l = len(train_id)
X_train = np.zeros((train_l, 224, 224, 3))
i = 0
for id in train_id:
  name = 'images_training_rev1/'+str(id)+'.jpg'
  X_train[i] = X_full[name]
  del X_full[name]
  i+=1
  
del train_l
del train_id

gc.collect()


22

In [0]:
test_l = len(test_id)  
X_test = np.zeros((test_l, 224, 224, 3))
i = 0  
for id in test_id:
  name = 'images_training_rev1/'+str(id)+'.jpg'
  X_test[i] = X_full[name]
  del X_full[name]
  i+=1

del test_id
del test_l
del X_full

gc.collect()

Y_train.drop(columns=["GalaxyID"])
Y_test.drop(columns=["GalaxyID"])

'''
At the end of this cell we have four important variables, namely: 
X_train and X_test --> containing the training and testing images as lists of images.
Y_train and Y_test --> containing the training and testing outputs for the images as pandas dataFrames.
'''

In [0]:
#Defining the hyperparameters
num_epochs = 500
batch_size = 50


In [0]:
#Building the ResNet Architecture

class ResNet:
  
  def __init__(self):
    pass
  
  def identity_block(self, X, f_shape, n_filters, stage, block_name):
    '''
    X -> Input tensor 
    f_shape -> Integer, denoting the shape of the middle filter
    n_filters -> List, denoting the number of filters in each layer
    stage -> Integer, denoting the name of the layer
    block_name -> Denotes the name of the layer
    '''
    
    
    f1, f2 = n_filters
    X_skip = X
    
    #1st component
    X = Conv2D(filters=f1, kernel_size=(f_shape, f_shape), strides=(1, 1), padding="same", name= "resnet"+str(stage)+block_name+"_branch_2a", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name = "batchnorm"+str(stage)+block_name+"_branch_2a")(X)
    X = Activation("relu")(X)
    
    #2nd component
    X = Conv2D(filters=f2, kernel_size=(f_shape, f_shape), strides=(1, 1), padding="same", name= "resnet"+str(stage)+block_name+"_branch_2b", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name = "batchnorm"+str(stage)+block_name+"_branch_2b")(X)
    X = Activation("relu")(X)
    
    #Inserting skip connection
    X = Add()([X, X_skip])
    X = Activation("relu")(X)
    
    return X
  
  def convolutional_block(self, X, f_shape, n_filters, stage, block_name, s=2):
    '''
    X -> Input tensor 
    f_shape -> Integer, denoting the shape of the middle filter
    n_filters -> List, denoting the number of filters in each layer
    stage -> Integer, denoting the name of the layer
    block_name -> Denotes the name of the layer
    s -> denoting the stride to be used
    '''
    
    f1, f2 = n_filters
    X_skip = X
    
    #1st component
    #1st component
    X = Conv2D(filters=f1, kernel_size=(f_shape, f_shape), strides=(s, s), padding="same", name= "resnet"+str(stage)+block_name+"_branch_2a", kernel_initializer=glorot_uniform(seed=0))(X) 
    X = BatchNormalization(axis=3, name = "batchnorm"+str(stage)+block_name+"_branch_2a")(X)
    X = Activation("relu")(X)
    
    #2nd component
    X = Conv2D(filters=f2, kernel_size=(1, 1), strides=(1, 1), padding="valid", name= "resnet"+str(stage)+block_name+"_branch_2b", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name = "batchnorm"+str(stage)+block_name+"_branch_2b")(X)
    X = Activation("relu")(X)
    
    #Convolution for the shortcut
    X_skip = Conv2D(filters=f2, kernel_size=(1, 1), strides=(s, s), padding="valid", name= "resnet"+str(stage)+block_name+"_branch_1", kernel_initializer=glorot_uniform(seed=0))(X_skip)
    X_skip = BatchNormalization(axis=3, name= "batchnorm"+str(stage)+block_name+"_branch_1")(X_skip)
    
    #Inserting skip connection
    X = Add()([X, X_skip])
    X = Activation("relu")(X)
    
    return X
  
  def ResNet18(self, input_shape=(224, 224, 3), classes=38):
    '''
    Defining the ResNet18 Architecture used
    
    Conv1 -> 7 X 7, 64, stride = 2, Output = (112 X 112)
    
    Conv2 -> 3 X 3 MaxPool, stride=2
             [3 X 3, 64] * 2
             
    Conv3 -> [3 X 3, 128] * 2
    
    Conv4 -> [3 X 3, 256] * 2
    
    Conv5 -> [3 X 3, 512] * 2
    '''
    
    X_input = Input(input_shape)
    X = ZeroPadding2D((3, 3))(X_input)
    
    #Conv1
    X = Conv2D(64, kernel_size=(7, 7), strides=(2, 2), name="conv1", kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name="batchnorm_conv1")(X)
    X = Activation("relu")(X)
    
    #Conv2
    X = MaxPooling2D((3, 3), strides = (2, 2))(X)
    X = self.convolutional_block(X, f_shape=3, n_filters=[64, 128], stage=2, block_name="a", s=2)
    X = self.identity_block(X, f_shape=3, n_filters=[64, 128], stage=2, block_name="b")
    
    #Conv3
    X = self.convolutional_block(X, f_shape=3, n_filters=[128, 256], stage=3, block_name="a", s=2)
    X = self.identity_block(X, f_shape=3, n_filters=[128, 256], stage=3, block_name="b")
    
    #Conv4
    X = self.convolutional_block(X, f_shape=3, n_filters=[256, 512], stage=4, block_name="a", s=2)
    X = self.identity_block(X, f_shape=3, n_filters=[256, 512], stage=4, block_name="b")
    
    #Conv5
    X = self.convolutional_block(X, f_shape=3, n_filters=[512, 1024], stage=5, block_name="a", s=2)
    X = self.identity_block(X, f_shape=3, n_filters=[512, 1024], stage=5, block_name="b")
    
    #Final Dense Layers
    X = AveragePooling2D((2, 2))(X)
    
    X = Flatten()(X)
    X = Dense(classes, activation="softmax", name="fc"+str(classes), kernel_initializer=glorot_uniform(seed=0))(X)
    
    #Create the model
    model = Model(inputs=X_input, outputs=X, name="ResNet18")
    
    return model

In [0]:
#Initializing and Compiling the model
Resnet18 = ResNet()
model = Resnet18.ResNet18(input_shape=(224, 224, 3), classes=38)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
train_tensorboard = TensorBoard(log_dir="./logs/train", histogram_freq = 0, write_graph  =True, write_images = True)
test_tensorboard = TensorBoard(log_dir="./logs/test", histogram_freq = 0, write_graph  =True, write_images = False)
checkpointer = ModelCheckpoint(filepath='tmp/weights_resnet18.hdf5', verbose=1, save_best_only = True)


Instructions for updating:
Colocations handled automatically by placer.


In [0]:
#Traing the model on the training data
model.fit(X_train, Y_train, epochs = 20, batch_size = 32, verbose = 1, callbacks = [checkpointer, train_tensorboard])

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
800/800 [==============================] - 8s 10ms/step - loss: 17069.5944 - acc: 0.9600
Epoch 2/20
 32/800 [>.............................] - ETA: 2s - loss: 84.6505 - acc: 1.0000

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


800/800 [==============================] - 3s 4ms/step - loss: 81.8365 - acc: 1.0000
Epoch 3/20
800/800 [==============================] - 3s 4ms/step - loss: 81.8365 - acc: 1.0000
Epoch 4/20
800/800 [==============================] - 3s 4ms/step - loss: 81.8365 - acc: 1.0000
Epoch 5/20
800/800 [==============================] - 3s 4ms/step - loss: 81.8365 - acc: 1.0000
Epoch 6/20
800/800 [==============================] - 3s 4ms/step - loss: 81.8365 - acc: 1.0000
Epoch 7/20
800/800 [==============================] - 3s 4ms/step - loss: 81.8365 - acc: 1.0000
Epoch 8/20
800/800 [==============================] - 3s 4ms/step - loss: 81.8365 - acc: 1.0000
Epoch 9/20
800/800 [==============================] - 3s 4ms/step - loss: 81.8365 - acc: 1.0000
Epoch 10/20
800/800 [==============================] - 3s 4ms/step - loss: 81.8365 - acc: 1.0000
Epoch 11/20
800/800 [==============================] - 3s 4ms/step - loss: 81.8365 - acc: 1.0000
Epoch 12/20
800/800 [============================

In [0]:
#Evaluating the data on the test set
prediction = model.evaluate(X_test, Y_test, batch_size = 32, verbose = 1)
print("Test Loss: "+str(prediction[0]))
print("Test Accuracy: "+str(prediction[1]))

201/201 [==============================] - 1s 5ms/step
Test Loss: 80.57517470174761
Test Accuracy: 1.0
